# 1. Election Data Cleaning and Restructuring
Electoin data come from the MIT election lab (https://electionlab.mit.edu/data) and are hosted by Harvard Dataverse (https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ/HEIJCQ&version=6.0). 



In [1]:
import pandas as pd
import numpy as np

In [2]:
#read in countypres csv
countypres = pd.read_csv('countypres_2000-2016.csv')

In [3]:
#remove any rows that have nan for state_po or candidatevotes (which are all erroneous)
countypres = countypres[~countypres['state_po'].isna()]
countypres = countypres[~countypres['candidatevotes'].isna()]

In [4]:
#only keep repulican and democrat candidates
major_party = ('democrat', 'republican')
countypres = countypres[countypres['party'].isin(major_party)]

In [5]:
#create a df with FIPS, county and state to reattach later
FIPS_data = countypres[['FIPS', 'state', 'county']].drop_duplicates()

#drop 'state', 'state_po', 'county', 'office', 'candidate', 'version' cols
countypres_lite = countypres.drop(['state', 'state_po', 'county', 'office', 'candidate', 'version'], axis = 1)

#set dtypes for FIPS and candidatevotes as int
countypres_lite[['FIPS', 'candidatevotes']] = countypres_lite[['FIPS', 'candidatevotes']].astype('int')

In [6]:
#create column 'vote_portion'  = 'candidatevotes' / 'totalvotes'
countypres_lite['vote_portion'] = countypres_lite['candidatevotes'] / countypres_lite['totalvotes']

#pivot countypres_lite to make vote portion for each party the values, and year/FIPS the index
countypres_lite_piv = pd.pivot_table(countypres_lite,
                                     values = 'vote_portion',
                                    index = ['year', 'FIPS'],
                                    columns = 'party')

#drop party and candidatevotes from countypres_lite
turnout = countypres_lite.drop(['party', 'candidatevotes', 'vote_portion'], axis = 1)
turnout.drop_duplicates(inplace = True)

#reset countypres_lite_piv index
countypres_lite_piv.reset_index(inplace = True)

#readd 'totalvotes' column
countypres_lite_piv = pd.merge(countypres_lite_piv, turnout, how = 'left', on = ['year', 'FIPS'])


In [7]:
#create a new column, 'rep_adv', containing republican - democrat for each row.
countypres_lite_piv['rep_adv'] = (countypres_lite_piv['republican'] - countypres_lite_piv['democrat'])*100

#drop the replublican and democrat columns
countypres_lite_piv.drop(['republican', 'democrat'], axis = 1, inplace = True)

#remerge in the FIPS data
countypres_lite_piv_cnames = countypres_lite_piv.merge(FIPS_data, how = 'left', on = 'FIPS')

# 2. Income Data Cleaning and Restructuring

Income data come from the following sources:

2000: https://www.census.gov/content/census/en/data/datasets/2000/demo/saipe/2000-state-and-county.html

2004: https://www.census.gov/content/census/en/data/datasets/2004/demo/saipe/2004-state-and-county.html

2008: https://www.census.gov/content/census/en/data/datasets/2008/demo/saipe/2008-state-and-county.html

2012: https://www.census.gov/content/census/en/data/datasets/2012/demo/saipe/2012-state-and-county.html

2016: https://www.census.gov/content/census/en/data/datasets/2016/demo/saipe/2016-state-and-county.html

In [8]:
#read in 2000 income .dat file
inc_00 = pd.read_table('2000_income.dat', sep = '\s+', usecols = [0, 1, 5, 20], header = None, names = ['state_fips', 'county_fips', 'pov_pct', 'med_inc'])

In [9]:
#drop any row where county_fips is 000 (these are statewide values)
inc_00 = inc_00[inc_00['county_fips'] != 0]

#add leading zeroes to county fips code to get them all to three digits
inc_00['county_fips'] = inc_00['county_fips'].apply(lambda x: '{0:0>3}'.format(x))

#concatenate the state and county codes to create full fips code, and drop the state and county codes
inc_00['FIPS'] = inc_00['state_fips'].map(str)+inc_00['county_fips'].map(str)
inc_00.drop(['state_fips', 'county_fips'], axis = 1, inplace = True)

#convert median income and FIPS to int type
inc_00[['med_inc', 'FIPS']] = inc_00[['med_inc', 'FIPS']].astype(int)

In [10]:
from glob import glob

#generate a list of the income csv files
income_list = glob('est*')
income_list

['est04all.csv', 'est08all.csv', 'est12all.csv', 'est16all.csv']

In [11]:
#create list of column names, cols
cols = ['state_fips', 'county_fips', 'pov_pct', 'med_inc']

#read in 2004-2016 income data
inc_04 = pd.read_csv(income_list[0], usecols = [0, 1, 7, 22], header = 0, names = cols)
inc_08 = pd.read_csv(income_list[1], usecols = [0, 1, 7, 22], header = 0, names = cols)
inc_12 = pd.read_csv(income_list[2], usecols = [0, 1, 7, 22], header = 0, names = cols)
inc_16 = pd.read_csv(income_list[3], usecols = [0, 1, 7, 22], header = 0, names = cols)

#create a list of the 04-16 income dfs
income_dfs = [inc_04, inc_08, inc_12, inc_16]

In [12]:
#drop NaNs, and drop any row where county_fips is 0 (these are statewide values)
for df in income_dfs:
    df = df.dropna()
    df = df[df['county_fips'] != 0]

In [13]:
#add leading zeroes to county fips code to get them all to three digits, and concatenate to create full fips code
for df in income_dfs:
    df['county_fips'] = df['county_fips'].apply(lambda x: '{0:0>3}'.format(x))
    df['FIPS'] = df['state_fips'].map(str)+df['county_fips'].map(str)
    df.drop(['state_fips', 'county_fips'], axis = 1, inplace = True)
    df[['FIPS']] = df[['FIPS']].astype(int)

In [14]:
#clean up the med_inc columns, dropna, and assign appropriate dtype
for df in income_dfs:
    df['med_inc'] = df['med_inc'].str.replace(",", "")
    df['med_inc'] = df['med_inc'].replace(".", np.nan)
    df.dropna(inplace = True)
    df[['med_inc']] = df[['med_inc']].astype(int)

In [15]:
#add a year column to each df
inc_00['year'] = 2000
inc_04['year'] = 2004
inc_08['year'] = 2008
inc_12['year'] = 2012
inc_16['year'] = 2016

#vertically concatenate all income dfs
for df in income_dfs:
    inc_00 = inc_00.append(df)

#reassign df to new generic var name
income = inc_00

In [16]:
#merge income df into countypres_lite_piv_cnames
countypres_income = pd.merge(countypres_lite_piv_cnames, income, how = 'left', on = ['year', 'FIPS'])

# 3. Population Data Restructuring and Cleaning

Data comes from the US Census Bureau: https://www2.census.gov/programs-surveys/popest/datasets/

In [17]:
#read in population CSVs
pop00_08 = pd.read_csv('cty_pop_00-08.csv', usecols = [0, 1, 4, 5, 6])
pop12_16 = pd.read_csv('cty_pop_12-16.csv', usecols = [0, 1, 4, 5])

#merge the two dfs on 'STATE' and 'COUNTY'
population = pd.merge(pop00_08, pop12_16, how = 'outer', on = ['STATE', 'COUNTY'])

#rename the columns
population.rename(columns = {'STATE':'state_fips', 
                             'COUNTY':'county_fips', 
                             'POPESTIMATE2000':'2000', 
                             'POPESTIMATE2004':'2004', 
                             'POPESTIMATE2008':'2008', 
                             'POPESTIMATE2012':'2012', 
                             'POPESTIMATE2016':'2016'}, 
                  inplace = True)

In [18]:
#drop all rows where county-fips is 0 (these are statewide values)
population = population[population['county_fips'] != 0]

#add leading zeroes to county fips code to get them all to three digits, and concatenate to create full fips code
population['county_fips'] = population['county_fips'].apply(lambda x: '{0:0>3}'.format(x))
population['FIPS'] = population['state_fips'].map(str)+population['county_fips'].map(str)
population.drop(['state_fips', 'county_fips'], axis = 1, inplace = True)
population['FIPS'] = population['FIPS'].astype(int)

population.set_index('FIPS', inplace = True)

In [19]:
#stack population so that year is vertical
pop_stack = population.stack()

#reset the index, rename the cols, and drop nas
pop_stack = pop_stack.reset_index()
pop_stack.rename(columns={'level_1':'year', 0:'population'}, inplace = True)
pop_stack.dropna(inplace = True)

#make 'year' int dtype
pop_stack['year'] = pop_stack['year'].astype(int)

In [20]:
#merge the pop_stack df into the countypres_income df
countypres_inc_pop = pd.merge(countypres_income, pop_stack, how = 'left', on = ['FIPS', 'year'])

In [21]:
#drop na vals from df
countypres_inc_pop.dropna(inplace = True)

#Make 'year' teh index col
countypres_inc_pop.set_index('year', inplace = True)

In [22]:
#make pov_pct type float
countypres_inc_pop['pov_pct'] = countypres_inc_pop['pov_pct'].astype(float) 

In [23]:
#create a new column for voter turnout percentage
countypres_inc_pop['turnout'] = (countypres_inc_pop['totalvotes'] / countypres_inc_pop['population'])*100

# 2. Data Visualization

In [88]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.palettes import brewer
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource, HoverTool, ColorBar
from bokeh.layouts import column, row, gridplot

In [101]:
#create list of years and and empty plots list
years = [2000, 2004, 2008, 2012, 2016]
plots = []

In [102]:
#create 5 plots with a for loop
for i in range(5):
    source = ColumnDataSource(data = {
    'x' : countypres_inc_pop.loc[years[i]].pov_pct,
    'y' : countypres_inc_pop.loc[years[i]].turnout,
    'rep_adv' : countypres_inc_pop.loc[years[i]].rep_adv,
    'county' : countypres_inc_pop.loc[years[i]].county,
    'state' : countypres_inc_pop.loc[years[i]].state,
    'population' : (countypres_inc_pop.loc[years[i]].population / 200000) + 1,
    'median income' : countypres_inc_pop.loc[years[i]].med_inc,
    'poverty' : countypres_inc_pop.loc[years[i]].pov_pct,
    'turnout' : countypres_inc_pop.loc[years[i]].turnout
    })
    #create linear colormap
    mapper = linear_cmap(field_name='rep_adv', palette=brewer['RdBu'][11] ,low=-100 ,high=100)
    #create plot
    plot = figure(title = '{} Presidential Election by County'.format(years[i]), 
              plot_height=400, 
              plot_width=700,
              x_range=(0, 35), 
              y_range=(10, 60))
    #add hover tool
    hover = HoverTool(tooltips=[('County', '@county'', ''@state'), 
                                ('Poverty (%)', '@poverty'),
                                ('GOP margin (%)', '@rep_adv')])
    plot.add_tools(hover)
    #add circle glyphs to the plot
    plot.circle(x='x', 
                y='y', 
                line_color = mapper, 
                color = mapper, 
                fill_alpha=0.4, 
                line_alpha = 0.6, 
                size = 'population', 
                source=source)
    #add colorbar
    color_bar = ColorBar(color_mapper=mapper['transform'], width=8,  location=(0,0))
    plot.add_layout(color_bar, 'right')
    # Set the x-axis label
    plot.xaxis.axis_label ='Population in Poverty (%)'
    # Set the y-axis label
    plot.yaxis.axis_label = 'Voter Turnout (%)'
    plots.append(plot)

In [103]:
#assign the plots to var names
p00, p04, p08, p12, p16 = plots

#tie the x and y ranges of each plot together
#p00.x_range = p04.x_range = p08.x_range = p12.x_range = p16.x_range
#p00.y_range = p04.y_range = p08.y_range = p12.y_range = p16.y_range

In [104]:
#create a gridplot with the 5 plots
layout = gridplot([
    [p00, p04],
    [p08, p12],
    [p16, None]
    ],
    toolbar_location = 'right')

output_file('election_plot.html')
show(layout)